In [ ]:
import h5py, sys, os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from datetime import datetime
from scipy.interpolate import griddata
print(os.getcwd())

%matplotlib widget

from jfunctions import *

# Get directory locations from param file
base_dir, run_dir, save_dir, version = read_params("./params.dat")

In [ ]:
#Load simulation data
b, phi_v, phi_c, u, v, w, times = load_data('th1_xz', 'th2_xz', 'th3_xz', 'u_xz', 'v_xz', 'w_xz')
NSAMP = len(times)

times, t0_idx = calibrate_time(save_dir, times)

b_az, w_az, phiv_az, phic_az = load_az_data('b_az', 'w_az', 'phiv_az', 'phic_az')

# Load simulation metadata
md = get_metadata()

# Load grids
gxf, gyf, gzf, dzf = get_grid(md)
gx, gy, gz, dz = get_grid(md, fractional_grid=False)

In [ ]:
print(md)

In [ ]:
#Create grid
X, Y = np.meshgrid(gx, gz)
Xf, Yf = np.meshgrid(gxf, gzf)

r_0 = md['r0']
dr = md['LX']/md['Nx']
nbins = int(md['Nx']/2)
r_bins = np.array([r*dr for r in range(0, nbins+1)])
r_points = np.array([0.5*(r_bins[i]+r_bins[i+1]) for i in range(nbins)])

Xa, Ya = np.meshgrid(r_bins, gz)
Xaf, Yaf = np.meshgrid(r_points, gzf)

print("Total time steps: %s"%NSAMP)
print("Dimensional times: ",times)


In [ ]:
# Environmental variables

alpha = md['alpha']
beta = md['beta']
q_0 = md['q0']
T0 = 300 # K

b_env = md['N2'] * (Yf - md['H'])
b_env[gzf < md['H']] = 0

b_az_env = md['N2'] * (Yaf - md['H'])
b_az_env[gzf < md['H']] = 0

T_env = 1340 * (b_env - beta * Yf)

phi_sat = q_0 * np.exp(alpha/1340 * T_env)

T = 1340 * (b - beta * Yf) + T0 - 273.15
theta = 1340 * b + T0

# Relative humidity
r = phi_v/phi_sat

# Total water
phi_t = phi_v + phi_c

In [ ]:
# Set up

contours_b = np.linspace(10, 150, 15) + T0
fn = 8
fnc = 1

steps = [t0_idx+16, t0_idx+32, len(times)-1]

xstart = get_index(0.2, gxf)-1
xend = get_index(0.4, gxf)+1
zstart = get_index(0.9*md['H'], gzf)-1
zend = get_index(1.5*md['H'], gzf)+1

Xfc, Yfc = np.meshgrid(gxf[xstart:xend], gzf[zstart:zend])
Xfi, Yfi = np.meshgrid(gxf[xstart:xend], np.linspace(gzf[zstart], gzf[zend], zend-zstart))

uc = u[:, zstart:zend, xstart:xend]
wc = w[:, zstart:zend, xstart:xend]
ui = np.zeros_like(uc)
wi = np.zeros_like(wc)

for i in range(NSAMP):
    ui[i] = griddata((Xfc.flatten(), Yfc.flatten()), uc[i].flatten(), (Xfi, Yfi))
    wi[i] = griddata((Xfc.flatten(), Yfc.flatten()), wc[i].flatten(), (Xfi, Yfi))

In [ ]:
fig_xz, ax_xz = plt.subplots(1,len(steps), constrained_layout=True, figsize=(12,3))

fig, ax = plt.subplots(1, 4, constrained_layout=True)

fig_p, ax_p = plt.subplots(1, 4, constrained_layout=True)

cols = plt.cm.viridis(np.linspace(0, 1, len(steps)))
for i,c in zip(range(len(steps)), cols):
    wvmr_im = ax_xz[i].pcolormesh(X, Y, r[steps[i]], cmap='YlGnBu')
    wvmr_im.set_clim(0, 5)
    theta_contour = ax_xz[i].contour(Xf, Yf, theta[steps[i]], colors='r', levels=contours_b)
    cloud_contour = ax_xz[i].contour(Xf, Yf, phi_t[steps[i]], colors='k', levels=[5e-4])
    flow = ax_xz[i].streamplot(Xfi[::fnc, ::fnc], Yfi[::fnc, ::fnc], ui[steps[i], ::fnc, ::fnc], wi[steps[i], ::fnc, ::fnc], 
                               broken_streamlines=True, density=1, minlength=0.1)#, maxlength=0.1)

    ax_xz[i].set_xlim(0.2, 0.4)
    ax_xz[i].set_ylim(0.9*md['H'], 1.5*md['H'])
    ax_xz[i].set_aspect(1)

    ax_xz[i].set_title("t = {0:.2f} s".format(times[steps[i]]))

    if i == 0:
        wvmr_cbar = fig_xz.colorbar(wvmr_im, label="wvmr", ax=ax_xz[:], location='bottom', shrink=0.7)

    #####
    # TODO try these as averages rather than vertical profiles

    ax_p[0].plot(theta[steps[i], :, int(md['Nx']/2)], gzf, color=c, label="t = {0:.2f} s".format(times[steps[i]]))

    ax_p[1].plot(T[steps[i], :, int(md['Nx']/2)], gzf, color=c)

    ax_p[2].plot(phi_v[steps[i], :, int(md['Nx']/2)], gzf, color=c)
    ax_p[2].plot(np.exp(alpha/1340 * T[steps[i], :, int(md['Nx']/2)]), gzf, ls='--', color=c)
    ax_p[2].set_xlim(0,.1)

    ax_p[3].plot(np.sqrt(u[steps[i], :, int(md['Nx']/2)]**2 + v[steps[i], :, int(md['Nx']/2)]**2), gzf, color=c)

    #####

    ew = []
    cloud = np.where((phic_az[steps[i]] + phiv_az[steps[i]]) < 5e-4, 0, 1)
    for j in range(md['Nz']):
        ew.append(r_points[np.min(np.argwhere(cloud[j] < 1))])

    ax[0].plot(ew, gzf, label="t = {0:.2f} s".format(times[steps[i]]), color=c)

    ax[1].plot(np.nanmean(np.where(cloud == 1, w_az[steps[i]], np.nan), axis=1), gzf, color=c)
    ax[1].plot(np.nanmax(np.where(cloud == 1, w_az[steps[i]], np.nan), axis=1), gzf, color=c)
    ax[1].plot(np.nanmin(np.where(cloud == 1, w_az[steps[i]], np.nan), axis=1), gzf, color=c)

    ax[2].plot(np.nanmean(np.where(cloud == 1, b_az[steps[i]]-b_az_env, np.nan), axis=1), gzf, color=c)

    ax[3].plot(np.nanmean(np.where(cloud == 1, phiv_az[steps[i]], np.nan), axis=1), gzf, color=c)
    ax[3].plot(np.nanmean(np.where(cloud == 1, phic_az[steps[i]], np.nan), axis=1), gzf, color=c, ls=':')
    ax[3].set_xscale('log')
    #ax[3].plot(np.nanmean(np.where(cloud == 1, phi_az[steps[i]], np.nan)))

ax_p[0].set_xlabel("Potential temperature")
ax_p[1].set_xlabel("Temperature")
ax_p[2].set_xlabel("Vapour concentration")
ax_p[3].set_xlabel("Wind speed")

ax_p[0].legend()
ax[0].legend()

ax[0].set_xlabel("Effective width")
ax[1].set_xlabel("Vertical velocity")
ax[2].set_xlabel("Buoyancy")
ax[3].set_xlabel("Vapour concentration")

for a in ax:
    a.set_ylim(0.15, 0.3)
    
for a in ax_p:
    a.set_ylim(0.15, 0.3)

plt.show()